In [2]:
import pandas as pd
import numpy as np
import requests
import json
import os
import psycopg2 # Needed to get database errors when uploading dataframe
import sql
import sqlalchemy
# hides warning messages
import warnings
warnings.filterwarnings("ignore")

# Original dictionary of competitors and locations

In [ ]:
search_term = {
'Air France Industrie KLM Engineering & Maintenance' : 
['China', 'United Kingdom', 'France', 'Netherland', 'USA', 'Turkey', 'Canada', 'Spain', 'Malaysia', 'Swiss', 'Denmark', 'Italy', 'Germany', 'Czech', 'Hungary', 'Singapore', 'Belgium', 'Dubai', 'Mexico'],
'Delta TechOps' : 
['China', 'United Kingdom', 'France', 'Netherland', 'USA', 'Turkey', 'Canada', 'Spain', 'Malaysia', 'Swiss', 'Denmark', 'Italy', 'Germany', 'Czech', 'Hungary', 'Singapore', 'Belgium', 'Dubai', 'Mexico'],
'Singapore Airlines Engineering Company' : 
['China', 'United Kingdom', 'France', 'Netherland', 'USA', 'Turkey', 'Canada', 'Spain', 'Malaysia', 'Swiss', 'Denmark', 'Italy', 'Germany', 'Czech', 'Hungary', 'Singapore', 'Belgium', 'Dubai', 'Mexico'],
'Turkish Technic' : 
['China', 'United Kingdom', 'France', 'Netherland', 'USA', 'Turkey', 'Canada', 'Spain', 'Malaysia', 'Swiss', 'Denmark', 'Italy', 'Germany', 'Czech', 'Hungary', 'Singapore', 'Belgium', 'Dubai', 'Mexico'],
'AJ Walter' : 
['China', 'United Kingdom', 'France', 'Netherland', 'USA', 'Turkey', 'Canada', 'Spain', 'Malaysia', 'Swiss', 'Denmark', 'Italy', 'Germany', 'Czech', 'Hungary', 'Singapore', 'Belgium', 'Dubai', 'Mexico'],
'SR Technics' : 
['China', 'United Kingdom', 'France', 'Netherland', 'USA', 'Turkey', 'Canada', 'Spain', 'Malaysia', 'Swiss'],
'ST Aerospace' : 
['Denmark', 'China'],
'StandardAero' : 
['France', 'USA'],
'AAR' : 
['Netherland', 'USA', 'Malaysia'],
'Airbus' : 
['United Kingdom', 'France', 'Spain', 'Italy', 'Germany'],
'Boeing' : 
['China', 'USA'],
'MTU' : 
['China', 'USA', 'Germany'],
'GE Aviation' : 
['Czech', 'Hungary', 'Singapore'],
'Rolls-Royce' : 
['United Kingdom', 'USA', 'Germany'],
'Raytheon Technologies' : 
['USA'],
'SAFRAN' : 
['United Kingdom', 'France', 'Germany', 'Belgium', 'Dubai', 'Mexico'],
'Nayak' : 
['Netherland', 'Italy', 'Germany']
}

In [ ]:
us_countries = {'AL' : 'Alabama', 'AK' : 'Alaska', 'AZ' : 'Arizona', 'AR' : 'Arkansas', 'CA' : 'California', 'CO' : 'Colorado', 'CT' : 'Connecticut', 
                'DE' : 'Delaware', 'FL' : 'Florida', 'GA' : 'Georgia', 'HI' : 'Hawaii', 'ID' : 'Idaho', 'IL' : 'Illinois', 'IN' : 'Indiana', 'IA' : 'Iowa', 
                'KS' : 'Kansas', 'KY' : 'Kentucky', 'LA' : 'Louisiana', 'ME' : 'Maine', 'MD' : 'Maryland', 'MA' : 'Massachusetts', 'MI' : 'Michigan', 
                'MN' : 'Minnesota', 'MS' : 'Mississippi', 'MO' : 'Missouri', 'MT' : 'Montana', 'NE' : 'Nebraska', 'NV' : 'Nevada', 'NH' : 'New Hampshire', 
                'NJ' : 'New Jersey', 'NM' : 'New Mexico', 'NY' : 'New York', 'NC' : 'North Carolina', 'ND' : 'North Dakota', 'OH' : 'Ohio', 'OK' : 'Oklahoma', 
                'OR' : 'Oregon', 'PA' : 'Pennsylvania', 'RI' : 'Rhode Island', 'SC' : 'South Carolina', 'SD' : 'South Dakota', 'TN' : 'Tennessee', 'TX' : 'Texas', 
                'UT' : 'Utah', 'VT' : 'Vermont', 'VA' : 'Virginia', 'WA' : 'Washington', 'WV' : 'West Virginia', 'WI' : 'Wisconsin', 'WY' : 'Wyoming'}

In [ ]:
from googletrans import Translator, constants
from geopy.geocoders import Nominatim
 
 
geolocator = Nominatim(user_agent = "geoapiExercises")
location = geolocator.geocode("Greater Munich Metropolitan Area")
print("Country Name: ", location)

In [ ]:
def us_states_to_country(jobs):
    us_list = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 
            'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 
            'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 
            'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 
            'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

    us_states = pd.Series([])

    for row in range(len(jobs)):

        if jobs['country'][row] in us_list:
            us_states[row] = jobs['country'][row]
            jobs['country'][row] = 'United States'
        

    jobs.insert(6, "us_state", us_states) 
    return jobs

In [12]:
en_cities = pd.read_csv('Data/worldcities.csv')

In [13]:
en_cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42905 entries, 0 to 42904
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   city        42905 non-null  object 
 1   city_ascii  42905 non-null  object 
 2   lat         42905 non-null  float64
 3   lng         42905 non-null  float64
 4   country     42905 non-null  object 
 5   iso2        42873 non-null  object 
 6   iso3        42905 non-null  object 
 7   admin_name  42810 non-null  object 
 8   capital     9812 non-null   object 
 9   population  42180 non-null  float64
 10  id          42905 non-null  int64  
dtypes: float64(3), int64(1), object(7)
memory usage: 3.6+ MB


In [14]:
en_cities.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6839,139.7744,Japan,JP,JPN,Tōkyō,primary,39105000.0,1392685764
1,Jakarta,Jakarta,-6.2146,106.8451,Indonesia,ID,IDN,Jakarta,primary,35362000.0,1360771077
2,Delhi,Delhi,28.6667,77.2167,India,IN,IND,Delhi,admin,31870000.0,1356872604
3,Manila,Manila,14.6000,120.9833,Philippines,PH,PHL,Manila,primary,23971000.0,1608618140
4,São Paulo,Sao Paulo,-23.5504,-46.6339,Brazil,BR,BRA,São Paulo,admin,22495000.0,1076532519


In [15]:
en_cities['state'] = en_cities['admin_name']

In [16]:
world_cities = en_cities[['city', 'state', 'lat', 'lng', 'country']]

In [17]:
world_cities.head()

,city,state,lat,lng,country
0,Tokyo,Tōkyō,35.6839,139.7744,Japan
1,Jakarta,Jakarta,-6.2146,106.8451,Indonesia
2,Delhi,Delhi,28.6667,77.2167,India
3,Manila,Manila,14.6000,120.9833,Philippines
4,São Paulo,São Paulo,-23.5504,-46.6339,Brazil


In [8]:
schema = os.getenv('schema')
from sql import engine

try:
    worldcities.to_sql(name='world_locations',      # Name of SQL table
                    con=engine,                     # Engine or connection
                    if_exists='replace',            # Drop the table before inserting new values 
                    schema=schema,                  # Use schmea that was defined earlier
                    index=False,                    # Write DataFrame index as a column
                    chunksize=50000,                 # Specify the number of rows in each batch to be written at a time
                    method='multi')                # Pass multiple values in a single INSERT clause
    print("'world_locations' was inserted successfully.")
except:
    print("world_cities_state was ignored.")

'world_locations' was inserted successfully.


In [4]:
worldcities = pd.read_csv('Data/worldcities.xlsx - Sheet1.csv')

In [7]:
worldcities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44102 entries, 0 to 44101
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   city     44102 non-null  object
 1   state    43873 non-null  object
 2   country  44102 non-null  object
dtypes: object(3)
memory usage: 1.0+ MB
